In [2]:
from binance import ThreadedWebsocketManager
from binance.client import Client
import pandas as pd
import numpy as np
import datetime

In [3]:
api_key = "DAX5KWi4IudakbEVvmQsjgqRYkSVycWQralfOuIWr7xSyQ4chqn192dLS4QGkG1q"  # TESTING testnet.binance.vision
api_secret = "i5YaSzFjlZTkrqsdMzEGUoukzkKehozhtNi0E5XPVP2UnbzyDuFB8RJuN1JMSTKZ"  # TESTING testnet.binance.vision

In [4]:
client = Client(api_key=api_key, api_secret=api_secret, testnet=True)

In [5]:
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'brokered': False,
 'updateTime': 1666462260922,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '9996.04744177', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [ ]:
class LongOnlyTrader():
    
    def __init__(self, symbol, interval, sma1, sma2, sma3 units=0.1, position=0):
        self.symbol = symbol
        self.interval = interval
        self.units = units
        self.position = position
        self.sma1 = sma1
        self.sma2 = sma2
        self.sma3 = sma3
        self.available_intervals = ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]

            #####Strategy attributs#####
        self.sma1 = sma1
        self.sma2 = sma2
        self.sma3 = sma3

    def start_trading(self, past_days=2):
        self.twm = ThreadedWebsocketManager()
        self.twm.start()
        
        if self.interval in self.available_intervals:
            self.get_recent_prices(past_days)
            self.twm.start_kline_socket(callback=self.stream_candles,
                                        symbol=self.symbol, interval=self.interval)
        # "else" to be added later in the course 
    
    def get_recent_prices(self, past_days):
        now = datetime.datetime.utcnow()
        past = str(now - datetime.timedelta(days=past_days))
        
        bars = client.get_historical_klines(self.symbol, self.interval, past)
        
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                      "Clos Time", "Quote Asset Volume", "Number of Trades",
                      "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
        df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
        df.set_index("Date", inplace=True)
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
        df["Complete"] = [True for row in range(len(df)-1)] + [False]
        
        self.data = df

    def stream_candles(self, msg):
        
        # extract the required items from msg
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        complete=       msg["k"]["x"]
    
        # print out
        print(".", end="", flush=True)
    
        # feed df (add new bar / update latest bar)
        self.data.loc[start_time] = [first, high, low, close, volume, complete]
        
        if complete:
            self.define_strategy()
            self.execute_order()

    def define_strategy(self):
        df = self.data.copy()
        df = df[["Open", "Close"]]
        # Define strategy  --  If last candle is green, buy. If red, sell
        cond = df.Open > df.Close  # Red candle
        
        df["Position"] = 1
        df.loc[cond, "Position"] = 0
        
        self.prepare_data = df.copy()
        
    def execute_order(self):
        if self.prepare_data["Position"].iloc[-1] == 1:  # Buy
            if self.position == 0:
                order = client.create_order(symbol=self.symbol, side="SELL", type="MARKET", quantity=self.units)
#                 order = client.create_order(symbol=self.symbol, side="BUY", type="MARKET", quantity=self.units)
                self.position = 1
                print("BUYING BITCOIN")

        if self.prepare_data["Position"].iloc[-1] == 0: # Sell:
            if self.position == 1:
#                 order = client.create_order(symbol=self.symbol, side="SELL", type="MARKET", quantity=self.units)
                order = client.create_order(symbol=self.symbol, side="BUY", type="MARKET", quantity=self.units)
                self.position = 0
                print("SELLING BITCOIN")